In [13]:
import boto3
from dotenv import load_dotenv
from datetime import datetime
import os
import pandas as pd

In [14]:
load_dotenv()

CSV_FILE_NAME_GENERATED_TODAY = f"rates-{datetime.today().date()}.csv".replace("-", "_")
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION_NAME = os.getenv('AWS_REGION_NAME')
AWS_S3_BUCKET_NAME = os.getenv('AWS_S3_BUCKET_NAME')

In [15]:
s3_client: boto3.client = boto3.client('s3',
                                           aws_access_key_id=AWS_ACCESS_KEY_ID,
                                           aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                           region_name=AWS_REGION_NAME)

In [16]:
import io
def get_file(file_name: str = CSV_FILE_NAME_GENERATED_TODAY) -> io.StringIO | None:
    try:
        print(file_name)
        response = s3_client.get_object(Bucket=AWS_S3_BUCKET_NAME, Key=file_name)
        csv_content = response['Body'].read().decode('utf-8')
        return io.StringIO(csv_content)
    except Exception as e:
        print(e)
        return None


In [17]:
rates_data:io.StringIO = get_file("data.csv")

data.csv


In [18]:
rates_data_df = pd.read_csv(rates_data, index_col=0, header=0)
rates_data_df

,symbols,countries,2023-08-13,2023-08-14
id,,,,
0,AED,United Arab Emirates Dirham,4.027006,4.014499
1,AFN,Afghan Afghani,92.483124,92.901544
2,ALL,Albanian Lek,103.792284,104.186004
3,AMD,Armenian Dram,426.368302,422.548045
4,ANG,Netherlands Antillean Guilder,1.981153,1.970852
...,...,...,...,...
165,YER,Yemeni Rial,274.475398,273.621753
166,ZAR,South African Rand,20.754214,20.814908
167,ZMK,Zambian Kwacha (pre-2013),9868.627272,9837.912389


In [19]:
# Obtenir les noms des colonnes de dates
date_columns = rates_data_df.columns[2:]

In [20]:
# Créer une liste vide pour stocker les lignes transformées
new_rates_data = []

In [21]:
# Parcourir chaque ligne du DataFrame d'origine
for index, row in rates_data_df.iterrows():
    symbols = row['symbols']
    countries = row['countries']

    # Parcourir les colonnes de dates
    for date_col in date_columns:
        date = date_col
        value = row[date_col]
        
        # Créer une nouvelle ligne pour chaque date
        new_row = [symbols, countries, date, value]
        
        # Ajouter la nouvelle ligne à la liste
        new_rates_data.append(new_row)

In [22]:
new_df = pd.DataFrame(new_rates_data, columns=['symbols', 'countries', 'date', 'value'])

In [23]:
new_df

,symbols,countries,date,value
0,AED,United Arab Emirates Dirham,2023-08-13,4.027006
1,AED,United Arab Emirates Dirham,2023-08-14,4.014499
2,AFN,Afghan Afghani,2023-08-13,92.483124
3,AFN,Afghan Afghani,2023-08-14,92.901544
4,ALL,Albanian Lek,2023-08-13,103.792284
...,...,...,...,...
335,ZMK,Zambian Kwacha (pre-2013),2023-08-14,9837.912389
336,ZMW,Zambian Kwacha,2023-08-13,20.963441
337,ZMW,Zambian Kwacha,2023-08-14,21.105351
338,ZWL,Zimbabwean Dollar,2023-08-13,353.030207
